ALL IMPORT

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ir_datasets
import os
import string
from nltk.corpus import wordnet
import nltk
from nltk import word_tokenize, PorterStemmer, pos_tag
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity

|INSTALL QUORA DATASET AND SAVE IT  USING PANDAS

In [ ]:

dataset = ir_datasets.load("beir/quora/dev")
data = {'id': [],'title': []}
for doc in dataset.docs_iter():
    print(doc)
    id = doc.doc_id
    text = doc.text
    data['id'].append(id)
    data['title'].append(text)
df = pd.DataFrame(data['title'], columns=['title'], index=data['id'])
df.to_csv('./test2.csv')

|INSTALL QUORA Dataset Query AND SAVE IT  USING PANDAS

In [ ]:
dataset = ir_datasets.load("beir/quora/dev")
data = {'id': [],'title': []}
for doc in dataset.queries_iter():
    print(doc)
    id = doc.query_id
    text = doc.text
    data['id'].append(id)
    data['title'].append(text)
df = pd.DataFrame(data,columns=['id', 'title'])
df.to_csv('./allQuery2.csv')

|INSTALL QUORA Dataset Qrels AND SAVE IT  USING PANDAS

In [ ]:

dataset = ir_datasets.load("beir/quora/dev")
data = {'query_id': [], 'doc_id': [], 'relevance': [], 'iteration': []}
for qrel in dataset.qrels_iter():
    data['query_id'].append(qrel.query_id)
    data['doc_id'].append(qrel.doc_id)
    data['relevance'].append(qrel.relevance)
    data['iteration'].append(qrel.iteration)

df = pd.DataFrame(data)
df.to_csv('./qrels.csv', index=False)

DATA PROCESSING CLASS

In [ ]:
stop_words = stopwords.words('english')
class TextProcessor:
    @staticmethod
    def lower(text):
         if isinstance(text, str):  # التحقق من أن القيمة من نوع str
            return text.lower()
         else:
            return ""
    
    @staticmethod
    def     tokenize_text(text):
        return word_tokenize(text)
    
    @staticmethod
    def remove_stopwords(text, stop_words=stop_words):
       return [word for word in text if not word in stop_words]
    

 
    @staticmethod
    def remove_punc(text):
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in text]
        # print("remove punctuation:", stripped)
        words = [word for word in stripped if word.isalpha()]
        return words
    
    @staticmethod
    def get_wordnet_pos(tag_parameter):
     tag = tag_parameter[0].upper()
     tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
     return tag_dict.get(tag, wordnet.NOUN)


    @staticmethod
    def lemetize_Text(text):
     pos_tags = nltk.pos_tag(text)
     lemmatizer = WordNetLemmatizer()
     lemmatized_words = [lemmatizer.lemmatize(text, pos=TextProcessor.get_wordnet_pos(pos_tag)) for text, pos_tag in pos_tags]
     return lemmatized_words
    
    @staticmethod
    def stemming(text):
        stemmer = PorterStemmer()
        return [stemmer.stem(word) for word in text]
    @staticmethod
    def process(text):
        text_processed = TextProcessor.lower(text)
        text_processed = TextProcessor.tokenize_text(text_processed)
        text_processed = TextProcessor.remove_stopwords(text_processed)
        text_processed = TextProcessor.remove_punc(text_processed)
        text_processed = TextProcessor.lemetize_Text(text_processed)
        text_processed = TextProcessor.stemming(text_processed)
        return text_processed

In [ ]:
df = pd.read_csv('./test2.csv') 

In [ ]:

def clean_title(title):
    processed_title = TextProcessor.process(title) 
    cleaned_title = ' '.join(processed_title)  
    return cleaned_title
df['title'] = df['title'].apply(clean_title) 
df.to_csv('./cleanedDataset3.csv', index=False) 

|FUNCTION FOR SAVE AND READ AND LOAD

In [ ]:
def read_documents(directory):
    documents = []
    document_names = []

    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r' ,encoding='utf-8') as file:
            document = file.read()
            documents.append(document)
            document_names.append(filename)

    return documents, document_names

def read_documents_from_one_file(filePath):
    df2 = pd.read_csv(filePath)
    documents2 = df2['title'].fillna('', inplace=False).tolist()
    document_names2 = df2['Unnamed: 0'].tolist()
    return documents2, document_names2

  

def save_vectorizer(vectorizer, path):
    with open(path, "wb") as f:
        pickle.dump(vectorizer, f)

def save_tfidf_matrix(tfidf_matrix, path):
    with open(path, "wb") as f:
        pickle.dump(tfidf_matrix, f)

def save_document_names(document_names, path):
    with open(path, "wb") as f:
        pickle.dump(document_names, f)

def save_inverted_index(inverted_index, path):
    with open(path, "wb") as f:
        pickle.dump(inverted_index, f)

def load_vectorizer(path):
    with open(path, "rb") as f:
        return pickle.load(f)
# 
def load_tfidf_matrix(path):
    with open(path, "rb") as f:
        return pickle.load(f)

def load_document_names(path):
    with open(path, "rb") as f:
        return pickle.load(f)

VECTORIZER TFIDF AND SAVE IT TO PICKLE FILES


In [ ]:
directory = "./cleanedDataset3.csv"
documents2, document_names2 = read_documents_from_one_file(directory)
print(documents2)
vectorizer = TfidfVectorizer()
print("Vectorizer Created.")
tfidf_matrix = vectorizer.fit_transform(documents2)
print("TF-IDF Created.")
feature_names = vectorizer.get_feature_names_out()
save_vectorizer(vectorizer, "C:\\Users\\Ghost\\Desktop\\pickleFiles\\vectorizer.pkl")
save_tfidf_matrix(tfidf_matrix, "C:\\Users\\Ghost\\Desktop\\pickleFiles\\tfidf_matrix.pkl")
save_document_names(document_names2, "C:\\Users\\Ghost\\Desktop\\pickleFiles\\document_names.pkl")

LOAD VICTORAIZE FILES AND MATCHING WITHOUT DOCUMENT CONTENT



In [ ]:
loaded_vectorizer = load_vectorizer("C:\\Users\\Ghost\\Desktop\\pickleFiles\\vectorizer.pkl")
loaded_tfidf_matrix = load_tfidf_matrix("C:\\Users\\Ghost\\Desktop\\pickleFiles\\tfidf_matrix.pkl")
loaded_document_names = load_document_names("C:\\Users\\Ghost\\Desktop\\pickleFiles\\document_names.pkl")                           

In [ ]:
def query_matching(query: str, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names):
    query2=TextProcessor.process(query)
    query3=' '.join(query2)
    query_tfidf = loaded_vectorizer.transform([query3])
    query_vector = query_tfidf
    similarity = cosine_similarity(query_vector, loaded_tfidf_matrix).flatten()
    # similarity1=np.argsort(similarity)[::-1]
    matching_results = []
    for i, doc_name in enumerate(loaded_document_names):
        if similarity[i] > 0:
            matching_results.append((doc_name, similarity[i]))
    sorted_results = sorted(matching_results, key=lambda x: x[1], reverse=True)
    return sorted_results

# query = "How does Quora look to a moderator"
# query2=TextProcessor.process(query)
# query3=' '.join(query2)
# print(query3)
# matching_results = query_matching(query3, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)
# for doc_name, similarity in matching_results:
#     print(f"Document ID: {doc_name}")
#     print(f"Similarity: {similarity}\n")

In [ ]:
def retrieve_files(query: str, file_path: str, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names):
    retrieved_files = []
    query_tfidf = loaded_vectorizer.transform([query])
    query_vector = query_tfidf.toarray()[0]
    similarity = cosine_similarity(query_vector.reshape(1, -1), loaded_tfidf_matrix).flatten()
    sorted_indices = np.argsort(similarity)[::-1]  # عكس ترتيب الفهارس للترتيب التنازلي
    sorted_documents = [(loaded_document_names[i], similarity[i]) for i in sorted_indices if similarity[i] > 0]

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(',')
            document = parts[0]
            content = ','.join(parts[1:])
            retrieved_files.append((document, content))

    merged_results = []
    for document, content in retrieved_files:
        for doc_name, doc_similarity in sorted_documents:
            if document == doc_name:
                merged_results.append((document, content, doc_similarity))
                break
    return merged_results
file_path = "./test2.csv"  # استبدله بمسار الملف الفعلي
results = retrieve_files("internet", file_path, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)
for document, content, similarity in results:
    print(f"Document: {document}")
    print(f"Content: {content}")
    print(f"Similarity: {similarity}\n")

Fetch Top 10 Sorted By Similarity

In [ ]:
def query_matching_top_10(query: str, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names):
    query2 = TextProcessor.process(query)
    query3 = ' '.join(query2)
    query_tfidf = loaded_vectorizer.transform([query3])
    query_vector = query_tfidf
    similarity = cosine_similarity(query_vector, loaded_tfidf_matrix).flatten()
    matching_results = []
    for i, doc_name in enumerate(loaded_document_names):
        if similarity[i] > 0:
            matching_results.append((doc_name, similarity[i]))
    sorted_results = sorted(matching_results, key=lambda x: x[1], reverse=True)
    
    # Return only the top 10 results
    return sorted_results[:10]

In [ ]:
matching=query_matching_top_10("step step guid invest share market india?", loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)
print(matching)

In [ ]:
def process_queries(file_path, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names):
    queries_df = pd.read_csv(file_path)
    doc_names_list = []
    query_id_results = {}
    for index, row in queries_df.iterrows():
        query_id = row['id']
        query = row['title'].strip()
        matching_results = query_matching_top_10(query, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)
        doc_names = [doc_name for doc_name, similarity in matching_results]
        doc_names_list.extend(doc_names)
        query_id_results[query_id] = {'Query': query, 'Document Names': doc_names}
    query_results_df = pd.DataFrame.from_dict(query_id_results, orient='index')
    query_results_df.to_csv('query_results_with_document_id3.csv', index_label='Query ID')
file_path = './allQuery2.csv'
process_queries(file_path, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)

Process All Query with Dataset And Save The Result

In [ ]:
def process_queries_without_query_text(file_path, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names):
    queries_df = pd.read_csv(file_path)
    doc_names_list = []
    query_id_results = {}
    for index, row in queries_df.iterrows():
        query_id = row['id']
        query = row['title'].strip()
        matching_results = query_matching_top_10(query, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)
        doc_names = [doc_name for doc_name, similarity in matching_results]
        doc_names_list.extend(doc_names)
        query_id_results[query_id] = {'doc_id': doc_names}
    query_results_df = pd.DataFrame.from_dict(query_id_results, orient='index')
    query_results_df.to_csv('query_with_doc_id_list', index_label='Query ID')
file_path = './allQuery2.csv'
process_queries_without_query_text(file_path, loaded_vectorizer, loaded_tfidf_matrix, loaded_document_names)

Calculate Precision  WITH TFIDF

In [ ]:
import csv
from typing import Dict

def calculate_precision_at_k(qrels_file: str, system_output_file: str, k=10):

    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    precision_at_k = {}
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        num_relevant = sum(1 for doc_id in doc_ids[:k] if str(doc_id) in relevant_docs)
        precision_at_k[query_id] = num_relevant / k
    
    return precision_at_k

# Example usage
precision_at_10 = calculate_precision_at_k('qrels.csv', './query_with_doc_id_list', k=10)
print(precision_at_10)

In [ ]:
Calculate Precision With Embedding Model Rresult

In [ ]:
import csv
from typing import Dict

def calculate_precision_at_k(qrels_file: str, system_output_file: str, k=10):

    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    precision_at_k = {}
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        num_relevant = sum(1 for doc_id in doc_ids[:k] if str(doc_id) in relevant_docs)
        precision_at_k[query_id] = num_relevant / k
    
    return precision_at_k

# Example usage
precision_at_10 = calculate_precision_at_k('qrels.csv', './query_result_without_cleaned_data.csv', k=10)
print(precision_at_10)

In [ ]:
import csv
from typing import Dict

def calculate_recall_at_k(qrels_file: str, system_output_file: str, k=10):
 
    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    recall_at_k = {}
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        total_relevant = sum(1 for doc_id, relevance in relevant_docs.items() if relevance > 0)
        num_relevant = sum(1 for doc_id in doc_ids[:k] if str(doc_id) in relevant_docs and relevant_docs[str(doc_id)] > 0)
        recall_at_k[query_id] = num_relevant / total_relevant
    
    return recall_at_k

# Example usage
recall_at_10 = calculate_recall_at_k('qrels.csv', './query_with_doc_id_list', k=10)
print(recall_at_10)

In [3]:
import csv
def calculate_map(qrels_file: str, system_output_file: str) -> float:
    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    average_precisions = []
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        total_relevant = sum(1 for doc_id, relevance in relevant_docs.items() if relevance > 0)
        
        if total_relevant == 0:
            average_precisions.append(0.0)
            continue
        
        precision_sum = 0
        num_relevant = 0
        for rank, doc_id in enumerate(doc_ids, start=1):
            if str(doc_id) in relevant_docs and relevant_docs[str(doc_id)] > 0:
                num_relevant += 1
                precision = num_relevant / rank
                precision_sum += precision
        
        average_precision = precision_sum / total_relevant
        average_precisions.append(average_precision)
    
    map_value = sum(average_precisions) / len(average_precisions)
    return map_value

# Example usage
map_value = calculate_map('qrels.csv', './query_with_doc_id_list')
print(f"MAP: {map_value}")

MAP: 0.6903602725211702


In [ ]:
import csv
def calculate_map(qrels_file: str, system_output_file: str) -> float:
    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    average_precisions = []
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        total_relevant = sum(1 for doc_id, relevance in relevant_docs.items() if relevance > 0)
        
        if total_relevant == 0:
            average_precisions.append(0.0)
            continue
        
        precision_sum = 0
        num_relevant = 0
        for rank, doc_id in enumerate(doc_ids, start=1):
            if str(doc_id) in relevant_docs and relevant_docs[str(doc_id)] > 0:
                num_relevant += 1
                precision = num_relevant / rank
                precision_sum += precision
        
        average_precision = precision_sum / total_relevant
        average_precisions.append(average_precision)
    
    map_value = sum(average_precisions) / len(average_precisions)
    return map_value

# Example usage
map_value = calculate_map('qrels.csv', './qoura_embiding_query_result2.csv')
print(f"MAP: {map_value}")

Calculate Map With Embedding MODEL

In [2]:
import csv
def calculate_map(qrels_file: str, system_output_file: str) -> float:
    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    average_precisions = []
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        total_relevant = sum(1 for doc_id, relevance in relevant_docs.items() if relevance > 0)
        
        if total_relevant == 0:
            average_precisions.append(0.0)
            continue
        
        precision_sum = 0
        num_relevant = 0
        for rank, doc_id in enumerate(doc_ids, start=1):
            if str(doc_id) in relevant_docs and relevant_docs[str(doc_id)] > 0:
                num_relevant += 1
                precision = num_relevant / rank
                precision_sum += precision
        
        average_precision = precision_sum / total_relevant
        average_precisions.append(average_precision)
    
    map_value = sum(average_precisions) / len(average_precisions)
    return map_value

# Example usage
map_value = calculate_map('qrels.csv', './query_result_with_embedding.csv')
print(f"MAP: {map_value}")

MAP: 0.8473709961427701


CALCULATE MRR WITH EMBEDDING MODEL

In [ ]:
import csv
def calculate_mrr(qrels_file: str, system_output_file: str) -> float:
    qrels = {}
    with open(qrels_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_id, relevance, _ = row
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    
    system_output = {}
    with open(system_output_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            query_id, doc_ids = row
            doc_id_list = [int(doc_id.strip()) for doc_id in doc_ids.strip('[]').split(', ') if doc_id.strip()]
            system_output[query_id] = doc_id_list
    
    mrr_values = []
    for query_id, doc_ids in system_output.items():
        relevant_docs = qrels.get(query_id, {})
        
        rank_of_first_relevant = float('inf')
        for rank, doc_id in enumerate(doc_ids, start=1):
            if str(doc_id) in relevant_docs and relevant_docs[str(doc_id)] > 0:
                rank_of_first_relevant = rank
                break
        
        if rank_of_first_relevant == float('inf'):
            mrr_values.append(0.0)
        else:
            mrr_values.append(1 / rank_of_first_relevant)
    
    mrr_value = sum(mrr_values) / len(mrr_values)
    return mrr_value

# Example usage
mrr_value = calculate_mrr('qrels.csv', './query_result_with_embedding.csv')
print(f"MRR: {mrr_value}")